Here we present the first example for the usage of pyiron-damask integration.
Within this notebook, user can define their own geometry files, material properties, and loading steps for the tensile test of the steel with different number of grains.

Basically, the inputs from the user's side are:
[x] Number of grains
[x] Size of the steel cubic
[x] Resoution of the cubic(the number of finite element grid in x,y,z direction)
[x] Elastic material parameter, i.e., Youngs modulus, poisson ratio...
[x] Loading steps for the tensile test

On the output side, pyiron-damask integration can offer:
[x] All the component of Cauchy stress, 1st Piola-Kirchoff stress
[x] All the comonent of strain tensor
[x] vonMises stress

In [12]:
# Here we import the basic components from pyiron-damask
from pyiron_base import Project
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
from pyiron_continuum.damask import damaskjob # this one is required for the pyiron-damask job

In [13]:
# Firstly, we need to create a project
pr = Project('tensile')
pr.remove_jobs_silently(recursive=True)

In [14]:
# Next, we need to tell pyiron, we want to create a damask type job
job = pr.create_job(damaskjob.DAMASK, 'damask_job')

In [15]:
# Now, lets set up the geometry size for the simulation
print(job.input.geometry['size'])

1e-05
